In [1]:
import pandas as pd
import numpy as np
import datetime as dt

df_clean = pd.read_csv('../data/processed/data_clean_uk.csv')

df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349199 entries, 0 to 349198
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   CustomerID   349199 non-null  int64         
 1   InvoiceNo    349199 non-null  int64         
 2   InvoiceDate  349199 non-null  datetime64[ns]
 3   TotalPrice   349199 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 10.7 MB


In [ ]:
last_order_date = df_clean['InvoiceDate'].max()
print(f"Data da última transação: {last_order_date.strftime('%Y-%m-%d')}")

snapshot_date = last_order_date + dt_timedelta(days=1)
print(f"Data de referência (Snapshot Date): {snapshot_date.strftime('%Y-%m-%d')}")

print(f"Dataset cobre o período de {df_clean['InvoiceDate'].min().strftime('%Y-%m-%d')} até {last_order_date.strftime('")